In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime

In [2]:
data = pd.read_csv("data/data_train/test_task_data.csv", sep=";")
data

,train_num,dep_st,ariv_st,price,dep_time,ariv_time
0,907,1909,1929,1790.87,00:50:00,20:00:00
1,908,1909,1929,1790.87,00:50:00,20:00:00
2,909,1929,1909,1790.87,10:01:00,01:00:00
3,909,1929,1921,199.15,11:10:00,01:00:00
4,910,1929,1909,1790.87,10:01:00,01:00:00
...,...,...,...,...,...,...
245,8501,1902,1937,195.00,05:50:00,21:50:00
246,8701,1902,1981,193.55,05:55:00,21:10:00
247,8801,1902,1981,193.40,06:35:00,19:30:00
248,8901,1937,1902,236.00,21:35:00,15:25:00


In [3]:
unique_stations = np.unique(data['dep_st'])
unique_stations

array([1902, 1909, 1921, 1929, 1937, 1981], dtype=int64)

In [4]:
data.sort_values(["dep_st", "ariv_st", "price"])

,train_num,dep_st,ariv_st,price,dep_time,ariv_time
58,1139,1902,1921,114.11,00:03:00,14:32:00
207,2001,1902,1921,139.98,20:10:00,12:30:00
249,9101,1902,1921,160.56,05:15:00,20:50:00
195,1812,1902,1921,161.35,21:35:00,14:25:00
199,1816,1902,1921,161.35,01:50:00,18:00:00
...,...,...,...,...,...,...
55,1126,1981,1921,314.35,11:20:00,17:36:00
109,1349,1981,1929,284.84,14:35:00,13:57:00
136,1405,1981,1929,330.59,17:40:00,16:21:00
70,1156,1981,1937,140.98,09:21:00,21:31:00


In [5]:
datetime_object = datetime.strptime(data["dep_time"][0], '%H:%M:%S')

In [6]:
def abs_value(a, b):
    diff = a - b
    return abs(diff.total_seconds())

data["dep_time"] = pd.to_timedelta(data["dep_time"])
data["ariv_time"] = pd.to_timedelta(data["ariv_time"])
data["time_diff"] = data.apply(lambda x: abs_value(x['ariv_time'], x['dep_time']), axis=1)
data.sort_values("time_diff")

,train_num,dep_st,ariv_st,price,dep_time,ariv_time,time_diff
108,1342,1921,1909,1421.12,0 days 13:44:00,0 days 13:52:00,480.0
118,1367,1921,1909,1421.12,0 days 11:21:00,0 days 11:11:00,600.0
109,1349,1981,1929,284.84,0 days 14:35:00,0 days 13:57:00,2280.0
110,1350,1929,1981,312.42,0 days 18:41:00,0 days 17:40:00,3660.0
111,1351,1929,1981,312.42,0 days 18:41:00,0 days 17:40:00,3660.0
...,...,...,...,...,...,...,...
182,1738,1902,1937,240.00,0 days 05:10:00,0 days 23:00:00,64200.0
183,1739,1902,1937,199.29,0 days 05:45:00,0 days 23:45:00,64800.0
223,3101,1902,1937,199.99,0 days 05:05:00,0 days 23:10:00,65100.0
1,908,1909,1929,1790.87,0 days 00:50:00,0 days 20:00:00,69000.0


In [51]:
d_price = {}
sorted_data = data.sort_values("price")
for item in unique_stations:
    d_price[item] = sorted_data[sorted_data["dep_st"] == item]
d_price

{1902:      train_num  dep_st  ariv_st   price        dep_time       ariv_time  \
 58        1139    1902     1921  114.11 0 days 00:03:00 0 days 14:32:00   
 10         919    1902     1981  130.60 0 days 04:59:00 0 days 20:06:00   
 82        1181    1902     1937  131.34 0 days 04:50:00 0 days 15:52:00   
 56        1137    1902     1937  131.34 0 days 19:53:00 0 days 08:21:00   
 72        1163    1902     1981  131.84 0 days 04:37:00 0 days 15:52:00   
 ..         ...     ...      ...     ...             ...             ...   
 8          917    1902     1929  300.18 0 days 09:25:00 0 days 20:01:00   
 217       2702    1902     1929  315.01 0 days 08:00:00 0 days 16:30:00   
 218       2703    1902     1929  315.01 0 days 08:00:00 0 days 16:30:00   
 216       2701    1902     1929  315.01 0 days 08:00:00 0 days 16:30:00   
 213       2203    1902     1929  319.76 0 days 13:15:00 0 days 17:55:00   
 
      time_diff  
 58     52140.0  
 10     54420.0  
 82     39720.0  
 56     

In [52]:
unique_stations


array([1902, 1909, 1921, 1929, 1937, 1981], dtype=int64)

In [69]:
from time import sleep
passed_stations = [1902]
path = []
passed_train_iter = {}

def find_path(d_price, cur_station=passed_stations[0]):
    while len(passed_stations) != len(unique_stations):
        print(passed_stations)
        possible_next_st = d_price[cur_station][~d_price[cur_station]['ariv_st'].isin(passed_stations)]
        print(possible_next_st)
        while possible_next_st.empty:
            print(f"Poped! {passed_stations.pop()}")
            cur_station = passed_stations[-1]
            print(f'Current station: {cur_station}')
            possible_next_st = d_price[cur_station][~d_price[cur_station]['ariv_st'].isin(passed_stations)]
            print(possible_next_st)
            path.pop()
           
        if cur_station in passed_train_iter:
            print(f"len(possible_next_st), passed_train_iter[cur_station]: {len(possible_next_st)} and {passed_train_iter[cur_station]}")
            if len(possible_next_st) - 1 == passed_train_iter[cur_station]:
                pst = passed_stations.pop()
                passed_train_iter[pst] = 0
                print(f"Poped! {pst}")
                print(f"Dict: {passed_train_iter}")
                cur_station = passed_stations[-1]
                print(f'Current station: {cur_station}')
                possible_next_st = d_price[cur_station][~d_price[cur_station]['ariv_st'].isin(passed_stations)]
                print(possible_next_st)
                path.pop()
            
#         print(d_price[cur_station])
        if cur_station not in passed_train_iter:
            passed_train_iter[cur_station] = 0
        else:
            passed_train_iter[cur_station] += 1
        
        print(f"Dict: {passed_train_iter}")
        train = possible_next_st.iloc[passed_train_iter[cur_station]]
        print(f"Num: {train.train_num}")
        
        passed_stations.append(train.ariv_st)
        cur_station = train.ariv_st
        print(f'Current station: {cur_station}')
        path.append(train)
#         sleep(1)
        
find_path(d_price)

[1902]
     train_num  dep_st  ariv_st   price        dep_time       ariv_time  \
58        1139    1902     1921  114.11 0 days 00:03:00 0 days 14:32:00   
10         919    1902     1981  130.60 0 days 04:59:00 0 days 20:06:00   
82        1181    1902     1937  131.34 0 days 04:50:00 0 days 15:52:00   
56        1137    1902     1937  131.34 0 days 19:53:00 0 days 08:21:00   
72        1163    1902     1981  131.84 0 days 04:37:00 0 days 15:52:00   
..         ...     ...      ...     ...             ...             ...   
8          917    1902     1929  300.18 0 days 09:25:00 0 days 20:01:00   
217       2702    1902     1929  315.01 0 days 08:00:00 0 days 16:30:00   
218       2703    1902     1929  315.01 0 days 08:00:00 0 days 16:30:00   
216       2701    1902     1929  315.01 0 days 08:00:00 0 days 16:30:00   
213       2203    1902     1929  319.76 0 days 13:15:00 0 days 17:55:00   

     time_diff  
58     52140.0  
10     54420.0  
82     39720.0  
56     41520.0  
72     

     train_num  dep_st  ariv_st    price        dep_time       ariv_time  \
65        1146    1929     1937   134.11 0 days 22:07:00 0 days 11:35:00   
66        1147    1929     1937   134.11 0 days 22:07:00 0 days 11:35:00   
141       1413    1929     1937   162.86 0 days 04:40:00 0 days 16:45:00   
230       5101    1929     1937   189.21 0 days 22:30:00 0 days 12:25:00   
234       5202    1929     1937   189.21 0 days 04:15:00 0 days 18:00:00   
233       5201    1929     1937   189.21 0 days 22:30:00 0 days 12:25:00   
231       5102    1929     1937   209.64 0 days 04:15:00 0 days 18:00:00   
232       5103    1929     1937   243.68 0 days 08:15:00 0 days 21:20:00   
110       1350    1929     1981   312.42 0 days 18:41:00 0 days 17:40:00   
111       1351    1929     1981   312.42 0 days 18:41:00 0 days 17:40:00   
2          909    1929     1909  1790.87 0 days 10:01:00 0 days 01:00:00   
119       1367    1929     1909  1790.87 0 days 11:21:00 0 days 00:30:00   
4          9

In [70]:
s = 0
i = 0
for item in path:
    s += item.price
    i += 1
    print (f"Num {i}: {item.train_num}")
s, path

Num 1: 1139
Num 2: 1013
Num 3: 1155
Num 4: 1405
Num 5: 1367


(2516.59,
 [train_num               1139
  dep_st                  1902
  ariv_st                 1921
  price                 114.11
  dep_time     0 days 00:03:00
  ariv_time    0 days 14:32:00
  time_diff            52140.0
  Name: 58, dtype: object,
  train_num               1013
  dep_st                  1921
  ariv_st                 1937
  price                 140.51
  dep_time     0 days 07:59:00
  ariv_time    0 days 18:03:00
  time_diff            36240.0
  Name: 30, dtype: object,
  train_num               1155
  dep_st                  1937
  ariv_st                 1981
  price                 140.51
  dep_time     0 days 03:30:00
  ariv_time    0 days 15:49:00
  time_diff            44340.0
  Name: 69, dtype: object,
  train_num               1405
  dep_st                  1981
  ariv_st                 1929
  price                 330.59
  dep_time     0 days 17:40:00
  ariv_time    0 days 16:21:00
  time_diff             4740.0
  Name: 136, dtype: object,
  train_num  